# Import Libraries

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## Analyze Transfer Data

In [ ]:
transfers = pd.read_csv('mimic-iii-clinical-database-demo-1.4/TRANSFERS.csv')


## Analyze Prescription Data


In [ ]:
prescriptions = pd.read_csv('mimic-iii-clinical-database-demo-1.4/PRESCRIPTIONS.csv')


## Analyze Lab Data

In [ ]:
lab_items = pd.read_csv('mimic-iii-clinical-database-demo-1.4/D_LABITEMS.csv')
lab_events = pd.read_csv('mimic-iii-clinical-database-demo-1.4/LABEVENTS.csv')


## Analyze Chart Events Data

Maybe use patient data to add a Gender Column to the chart_events data, and then create a dictionary for each itemid -> Label for D_Items. And then plot the events data for each gender.

In [ ]:
chart_events = pd.read_csv('mimic-iii-clinical-database-demo-1.4/CHARTEVENTS.csv')
d_items = pd.read_csv('mimic-iii-clinical-database-demo-1.4/D_ITEMS.csv')

## Analyze Procedures Data

This is very similar to Diagnosis Data Analysis from Tutorial

In [ ]:
procedures = pd.read_csv('mimic-iii-clinical-database-demo-1.4/PROCEDURES_ICD.csv')
procedures_labels = pd.read_csv('mimic-iii-clinical-database-demo-1.4/D_ICD_PROCEDURES.csv')